In [18]:
import cv2
from tqdm import tqdm
import os

def process_video(input_video_path, output_video_path):
    # Khởi tạo đối tượng VideoCapture
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Lấy tổng số frames và FPS của video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / original_fps
    # Tính toán phần trăm và thời điểm bắt đầu, kết thúc để cắt
    if duration > 110:
        start_percent = 20
        end_percent = 80
    elif duration > 60:
        start_percent = 15
        end_percent = 85
    elif duration > 10:
        start_percent = 10
        end_percent = 90
    else:
        start_percent = 0
        end_percent = 100

    start_frame = int((start_percent / 100) * total_frames)
    end_frame = int((end_percent / 100) * total_frames)

    # Lấy kích thước frame của video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Tạo đối tượng VideoWriter với FPS là desired_fps
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, original_fps, (224, 224))

    # Đọc từng frame và ghi frame vào video mới nếu nằm trong khoảng đã tính và theo tỷ lệ FPS mới
    current_frame = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Chỉ xử lý frame nếu nằm trong khoảng thời gian đã định
        if start_frame <= current_frame <= end_frame:
            # Resize frame về kích thước 224x224
            frame = cv2.resize(frame, (224, 224))
            out.write(frame)

        current_frame += 1

    # Giải phóng tài nguyên
    cap.release()
    out.release()

    print("Video đã được xử lý và lưu tại: ", output_video_path)

In [19]:
data_dir = "video"
output_dir = "output"

for className in tqdm(os.listdir(data_dir)):
    class_dir = os.path.join(data_dir, className)
    if os.path.isdir(class_dir) and not className.startswith('.'):  # Bỏ qua các thư mục ẩn
        for video_file in os.listdir(class_dir):
            if not video_file.startswith('.'):  # Bỏ qua các tệp ẩn
                input_video_path = os.path.join(class_dir, video_file)
                video_name = video_file.split('.')[0]
                output_video_path = f"{output_dir}/{className}/{video_name}.mp4"
                os.makedirs(f"{output_dir}/{className}", exist_ok=True)

                # Kiểm tra xem video đã tồn tại trong thư mục đầu ra hay không
                if not os.path.exists(output_video_path):
                    # Đọc độ dài của video
                    cap = cv2.VideoCapture(input_video_path)
                    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    fps = int(cap.get(cv2.CAP_PROP_FPS))
                    duration = frame_count / fps  # Độ dài video tính theo giây

                    # Kiểm tra nếu độ dài video lớn hơn 300 giây, thì bỏ qua
                    if duration > 300:
                        print(f"Video {input_video_path} có độ dài lớn hơn 5 phút và đã bị bỏ qua.")
                        continue  # Bỏ qua video này và chuyển sang video tiếp theo
                    else:
                        print(f"Xử lý video {input_video_path}")
                        process_video(input_video_path, output_video_path)
                else:
                    print(f"Video {output_video_path} đã được xử lý trước đó và tồn tại trong thư mục đầu ra.")

 50%|█████     | 1/2 [00:00<00:00,  7.19it/s]

Xử lý video video\a\output.mp4
Video đã được xử lý và lưu tại:  output/a/output.mp4

Xử lý video video\dp\russian_twist_17.mp4


100%|██████████| 2/2 [00:01<00:00,  1.06it/s]

Video đã được xử lý và lưu tại:  output/dp/russian_twist_17.mp4

